# 筛股票

In [1]:

import pandas as pd

# 转化hdf5格式数据到DataFrame
def read_data(name):
    file = name + '.h5'
    store = pd.HDFStore(file, mode='r')
    data = store.select('data')
    store.close()

    ticker_list = [ticker for ticker in data.columns if ticker[0] in ['0', '3', '6']]
    output = data[ticker_list]
    return output

# 更换目录到数据文件夹
import os
os.getcwd()
os.chdir('/Users/Zhang/Documents/华泰Project/数据/数据/估值')

# 节选回测区间
def date(df):
    z = df.loc['2012-02-09':'2019-05-16']
    return z

# 因为拥有的个股不一样，所以要先去掉另一个多出来的个股，然后进行两个DataFrame之间除法运算
# Return出来的数据类型是DataFrame
def value(df1,df2):
    if len(df1.columns) > len(df2.columns):
        columns = df1.columns.difference(df2.columns)
        df3 = df1.drop(columns,axis=1)
        value = pd.DataFrame(df3.values/df2.values,columns = df3.columns,index = df3.index)
    elif len(df1.columns) < len(df2.columns):
        columns = df2.columns.difference(df1.columns)
        df3 = df2.drop(columns,axis=1)
        value = pd.DataFrame(df1.values/df3.values,columns = df3.columns,index = df3.index) 
    else:
        value = pd.DataFrame(df1.values/df2.values,columns = df1.columns,index = df1.index)  
    return value

# 减少运算数据量，在因子后面_1来表示我们slice了data
def slice(df):
    df_1 = df.iloc[:,:300]
    return df_1

# 根据华泰金工研报的因子暴露度计算，是每个自然月的最后一个交易日进行计算，原因不知道？
# 不明白为什么不可以自己每日的因子暴露度，准确率高，由于市值 = market price*# of shares outstanding。
# market price of stock 每日都在变
# 所以先按研报算法为准，我们先去除 除了自然月最后一个交易日的因子们


# 寻找每个月最后一个交易日

year = [2012,2013,2014,2015,2016,2017,2018,2019] ## Change year
month = [1,2,3,4,5,6,7,8,9,10,11,12]

Price = date(read_data("closePrice"))
Price['year'] = pd.DatetimeIndex(Price.index).year#['tradeDate'])#.year
Price['month'] = pd.DatetimeIndex(Price.index).month
Price['day'] = pd.DatetimeIndex(Price.index).day
list_1 = []

def return_list(j):
    Find_month = Find_year.loc[Find_year['month'] == j]
    z = Find_month.iloc[-1,:].name
    list_1.append(z)
    return list_1

# since we start from Feb 2014 and end in May 2019
for i in year:
    Find_year = Price.loc[Price['year'] == i]
    if i == 2012:
        for j in month[1:]:
            list_1 = return_list(j)
    elif i == 2019:
        for j in month[:4]:
            list_1 = return_list(j)
    else:
        for j in month:
            list_1 = return_list(j)
            
# create the function that will obtain data from every last trading day of month
def month_end(df):
    df1 = pd.DataFrame()
    for i in list_1:
        df_s = df.loc[df.index == i]
        df1 = pd.concat([df1,df_s],axis = 0)
    return df1

ImportError: Unable to import required dependencies:
numpy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the multiarray numpy extension module failed.  Most
likely you are trying to import a failed build of numpy.
Here is how to proceed:
- If you're working with a numpy git repository, try `git clean -xdf`
  (removes all files not under version control) and rebuild numpy.
- If you are simply trying to use the numpy version that you have installed:
  your installation is broken - please reinstall numpy.
- If you have already reinstalled and that did not fix the problem, then:
  1. Check that you are using the Python you expect (you're using /Users/Zhang/anaconda3/bin/python),
     and that you have no directories in your PATH or PYTHONPATH that can
     interfere with the Python and numpy versions you're trying to use.
  2. If (1) looks fine, you can open a new issue at
     https://github.com/numpy/numpy/issues.  Please include details on:
     - how 

# 估值类因子

## EP 净利润(TTM)/总市值

In [ ]:
NI = date(read_data("NIncome"))
MV = date(read_data("marketValue"))
EP = value(NI,MV)
EP.index = NI.index
EP.to_csv(r'/Users/Zhang/Documents/华泰Project/Computed_Factor/Fundamental/EP.csv')
#减少运算数据量，在因子后面_1来表示我们slice了data
EP_1 = slice(EP)
EP_1 = month_end(EP_1)
EP_1.to_csv(r'/Users/Zhang/Documents/华泰Project/Computed_Factor/Fundamental/EP_1.csv')

## 市值

In [ ]:
MV_1 = slice(MV)
MV_1 = month_end(MV_1)
MV_1.to_csv(r'/Users/Zhang/Documents/华泰Project/Computed_Factor/Fundamental/MV_1.csv')

## EPcut 扣除非经常性损益后净利润(TTM)/总市值

In [ ]:
NiAttr = date(read_data("niAttrPCut"))
EPcut = value(NiAttr,MV)
EPcut.to_csv(r'/Users/Zhang/Documents/华泰Project/Computed_Factor/Fundamental/EPcut.csv')
EPcut_1 = month_end(slice(EPcut))
EPcut_1.to_csv(r'/Users/Zhang/Documents/华泰Project/Computed_Factor/Fundamental/EPcut_1.csv')

## BP（净资产=所有者权益TTM）净资产/总市值

In [ ]:
Tsh = date(read_data("TShEquity"))
BP = value(Tsh,MV)
BP.to_csv(r'/Users/Zhang/Documents/华泰Project/Computed_Factor/Fundamental/BP.csv')
BP_1 = month_end(slice(BP))
BP_1.to_csv(r'/Users/Zhang/Documents/华泰Project/Computed_Factor/Fundamental/BP_1.csv')

## SP 营业收入(TTM)/总市值

In [ ]:
rev = date(read_data("revenue"))
SP = value(rev,MV)
SP_1 = month_end(slice(SP))
SP_1.to_csv(r'/Users/Zhang/Documents/华泰Project/Computed_Factor/Fundamental/SP_1.csv')

## NCFP 净现金流(TTM)/总市值

In [ ]:
NCE = date(read_data("NCEEndBal"))
NCFP = value(NCE,MV)
NCFP_1 = month_end(slice(NCFP))
NCFP_1.to_csv(r'/Users/Zhang/Documents/华泰Project/Computed_Factor/Fundamental/NCFP_1.csv')

## OCFP 经营性现金流(TTM)/总市值

In [ ]:
NCF = date(read_data("NCFOperateA"))
OCFP = value(NCF,MV)
OCFP_1 = month_end(slice(OCFP))
OCFP_1.to_csv(r'/Users/Zhang/Documents/华泰Project/Computed_Factor/Fundamental/OCFP_1.csv')
# 无Dividends数据

## G/PE 净利润(TTM)同比增长率/PE_TTM

In [6]:
import numpy as np
G = date(read_data("niYoY"))
G = G.replace(0,np.nan)
EPS = date(read_data("EPS"))
EPS = EPS.replace(0,np.nan) 
# EPS.eq(0).any().any()
Price = date(read_data("closePrice"))
PE = value(Price,EPS)
G_PE = value(G,PE)
G_PE_1 = month_end(slice(G_PE))
G_PE_1.to_csv(r'/Users/Zhang/Documents/华泰Project/Computed_Factor/Fundamental/G_PE_1.csv')

In [ ]:
import os
import pandas as pd
#In_captial = np.log(MV)
os.chdir('/Users/Zhang/Documents/华泰Project/数据/数据/raw_data/raw_data/')
Index = pd.read_csv("上证综指.csv")

# 整理Stock和Index Return

## 整理Stock Return

In [ ]:
Index = Index[['TRADE_DT','S_DQ_CLOSE']]
Index['TRADE_DT'] = pd.to_datetime(Index['TRADE_DT'],format = '%Y%m%d',errors = 'ignore')
Index.set_index('TRADE_DT',inplace=True)
Stock_Return = Price.pct_change().iloc[1:]
Stock_Return = Stock_Return.fillna(0)
Stock_Return.index = pd.to_datetime(pd.Series(Stock_Return.index))# change dtype of object to datetime
Stock_Return.to_csv(r'/Users/Zhang/Documents/华泰Project/Computed_Factor/Fundamental/Stock_Return.csv')
Stock_Return_1 = month_end(slice(Stock_Return))
Stock_Return_1.to_csv(r'/Users/Zhang/Documents/华泰Project/Computed_Factor/Fundamental/Stock_Return_1.csv')

## 整理Index DataFrame

In [7]:
# Index return
Index_Return = Index['S_DQ_CLOSE'].pct_change().iloc[1:]
Index_Return.to_csv(r'/Users/Zhang/Documents/华泰Project/Computed_Factor/Fundamental/Index_Return.csv')
Index_Return_1 = month_end(Index_Return.loc['2012-02-09':'2019-05-16'])
Index_Return_1.to_csv(r'/Users/Zhang/Documents/华泰Project/Computed_Factor/Fundamental/Index_Return_1.csv')

/Users/Zhang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


# 成长类因子

In [ ]:
os.getcwd()
os.chdir('/Users/Zhang/Documents/华泰Project/数据/数据/成长')

## 营业收入(最新财报，YTD)同比增长率(%)

In [ ]:
revenue = date(read_data("revenue"))
Sales_G_q = revenue
Sales_G_q_1 = month_end(slice(Sales_G_q))

## 净利润(最新财报，YTD)同比增长率(%)

In [ ]:
Profit_G_q = date(read_data("niYoY"))
Profit_G_q_1 = month_end(slice(Profit_G_q))

## 经营性现金流(最新财报，YTD)同比增长率

In [ ]:
OCF_G_q = date(read_data("NCFOperateA"))
OCF_G_q_1 = month_end(slice(OCF_G_q))

## ROE(最新财报，YTD)同比增长率

In [62]:
ROE_G_q = date(read_data("ROEYOY"))
ROE_G_q_1 = month_end(slice(ROE_G_q))

# 财务质量类因子

In [ ]:
os.getcwd()
os.chdir('/Users/Zhang/Documents/华泰Project/数据/数据/财务质量')

## ROE(最新财报，TTM)

In [ ]:
ROE = date(read_data('ROE'))
ROE_1 = month_end(slice(ROE))

## ROA(最新财报，TTM)

In [ ]:
ROA = date(read_data('ROA'))
ROA_1 = month_end(slice(ROA))

## 毛利率(最新财报，TTM)

In [ ]:
grossprofitmargin_ttm = date(read_data('grossMARgin'))
grossprofit_1 = month_end(slice(grossprofitmargin_ttm))

## 扣除非经常性损益后净利润率(最新财报，TTM)

In [ ]:
profitmargin_ttm = date(read_data('npMARgin'))
profitmargin_1 = month_end(slice(profitmargin_ttm ))

## 资产周转率(最新财报，TTM)

In [ ]:
assetturnover_ttm = date(read_data('taTurnover'))
assetturnover_1 = month_end(slice(assetturnover_ttm))

## 经营性现金流/净利润(最新财报，TTM)

In [21]:
operationcashflowratio_ttm = value(NCF,NI)
ocf_1 = month_end(slice(operationcashflowratio_ttm ))

/Users/Zhang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


# 杠杆类因子

In [ ]:
os.chdir('/Users/Zhang/Documents/华泰Project/数据/数据/杠杆')

## 总资产/净资产

In [63]:
TAssets = date(read_data('TAssets'))
TAssets_1 = month_end(slice(TAssets))
Tsh_1 = month_end(slice(Tsh)) #净资产

for i in range(TAssets_1.shape[1]):
    financial_lev_list = []
    a = TAssets_1.iloc[:,i].values
    b = Tsh_1.iloc[:,i].values
    c = np.divide(a,b,out = np.zeros_like(a),where=b!=0) # divide cal anywhere 'where' b!=0
    #financial_lev_list.append(c)
    if i == 0:
        financial_lev_df = pd.Series(c)
    else:
        c = pd.Series(c)
        financial_lev_df = pd.concat([financial_lev_df,c],axis = 1)

financial_lev_df.index = TAssets_1.index
financial_lev_df.columns = TAssets_1.columns       


## debtequityratio 非流动负债/净资产

In [23]:
TNCL = date(read_data('TNCL'))
TNCL_1 = month_end(slice(TNCL))

for i in range(TNCL_1.shape[1]):
    TNCL_list = []
    a = TNCL_1.iloc[:,i].values
    b = Tsh_1.iloc[:,i].values
    c = np.divide(a,b,out = np.zeros_like(a),where=b!=0) # divide cal anywhere 'where' b!=0
    #financial_lev_list.append(c)
    if i == 0:
        debt_equity_df = pd.Series(c)
    else:
        c = pd.Series(c)
        debt_equity_df = pd.concat([debt_equity_df,c],axis = 1)

debt_equity_df.index = TNCL_1.index
debt_equity_df.columns = TNCL_1.columns    
debt_equity_df.head()
# cashratio 现金比率 暂无数据

# currentratio 流动比率 暂无数据

/Users/Zhang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


,000001,000002,000003,000004,000005,000006,000007,000008,000009,000010,...,000712,000713,000715,000716,000717,000718,000719,000720,000721,000722
2014-02-28,NaN,0.349106,-0.210472,0.004977,0.000511,0.646285,NaN,NaN,0.262816,0.002747,...,0.318653,0.021213,0.010450,0.142776,0.249173,0.790872,0.002293,1.857254,0.018263,0.065855
2014-03-31,NaN,0.425306,-0.210472,0.004977,0.000511,0.645327,NaN,NaN,0.262816,0.002747,...,0.466298,0.021213,0.011411,0.142776,0.249173,0.790872,0.002293,1.857254,0.018263,0.065677
2014-04-30,NaN,0.424400,-0.211586,0.227034,0.000582,0.552894,NaN,NaN,0.409105,0.001330,...,0.293525,0.026675,0.021760,0.070466,0.102810,1.264583,0.000357,2.246931,0.013500,0.060351
2014-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-06-30,NaN,0.424400,-0.211586,0.227034,0.000582,0.552894,NaN,NaN,0.409105,0.001330,...,0.293525,0.025619,0.021760,0.070466,0.102810,1.264583,0.000357,2.246931,0.013500,0.060351


# 动量反转因子

## 个股 60 个月收益与上证综指回归的截距项与Beta

In [ ]:
import time
from sklearn.linear_model import LinearRegression
start_time = time.time()
cons = np.ones((60,1)).reshape(-1,1)
Stock_Return_1 = Stock_Return_1.fillna(0)
Index_Return_1 = Index_Return_1.fillna(0)
beta_Df = pd.DataFrame()
alpha_Df = pd.DataFrame()
#print('Beta_df shape'+str(beta_Df))
for i in range(Stock_Return_1.shape[1]):
    beta = []
    alpha = []
    beta_Series = pd.Series()
    alpha_Series = pd.Series()
    for j in range((Stock_Return_1.shape[0])-60): 
        S_R = np.array(Stock_Return_1.iloc[j:60+j,i]).reshape(-1,1)
        X = np.concatenate((S_R,cons),axis=1)
        Y = np.array(Index_Return_1.iloc[j:60+j])
        reg = LinearRegression().fit(X,Y)
        beta.append(reg.coef_[0])
        alpha.append(reg.intercept_)
        
    beta_Series = pd.Series(beta)
    #print('beta series shape' + str(beta_Series.shape))

    alpha_Series = pd.Series(alpha)
    #print('alpha series shape' + str(alpha_Series.shape))
    beta_Df = pd.concat([beta_Df,beta_Series],axis = 1)
    alpha_Df = pd.concat([alpha_Df,alpha_Series],axis = 1)

beta_Df.index = Stock_Return_1.iloc[60:].index
alpha_Df.index = Stock_Return_1.iloc[60:].index # reset index + check number of index
beta_Df.columns = Stock_Return_1.columns 
alpha_Df.columns = Stock_Return_1.columns

print("--- %s seconds ---" % (time.time() - start_time))
#np.array(beta).reshape(-1,1)

## 截距项（HAlpha）

In [ ]:
alpha_Df.to_csv(r'/Users/Zhang/Documents/华泰Project/Computed_Factor/Fundamental/Halpha.csv')

## Beta

In [65]:
beta_Df.to_csv(r'/Users/Zhang/Documents/华泰Project/Computed_Factor/Fundamental/beta.csv')

--- 1.5961060523986816 seconds ---


## 个股最近N 个月收益率，N=1，3，6，12

In [66]:
def return_m(df,m):
    dff = pd.DataFrame()
    #Nested_list = []
    for i in range(df.shape[1]):
        list_1 = []
        Series = pd.Series() # need to reinitialize
        for j in range(df.shape[0]-m):
            n = df.iloc[j:m+j,i].cumprod().iloc[-1]
            list_1.append(n)
        #Nested_list.append(list_1)
        #dff = pd.DataFrame(Nested_list)
        Series = pd.Series(list_1)
        dff = pd.concat([dff,Series],axis = 1)       
   
    dff.index = df.iloc[m:].index # reset index + check number of index
    dff.columns = df.columns # reset columns + check number of columns
    return dff

return_1 = return_m(Stock_Return_1,1)
#return_3 = return_m(Stock_Return_1,3)
#return_6 = return_m(Stock_Return_1,6)
#return_12 = return_m(Stock_Return_1,12)

In [68]:
# grouby turnover dataframe by stock
### 如果数据源都呈一列，使用这个cell可以整理数据，纵轴为个股，横轴为时间
### 用来转换raw data
# 改变turnover到你想要的csv
os.chdir('/Users/Zhang/Documents/华泰Project/数据/数据/raw_data/raw_data/')
turnover = pd.read_csv("换手率.csv")
turnover['mddate'] = pd.to_datetime(turnover['mddate'],format = '%Y%m%d',errors = 'ignore')
print(type(turnover.stock.unique()))

s = turnover.groupby(['stock']) 

def stock_i(df): # function to modify df
    stock_name = df.iloc[0]['stock']
    new_df = df.rename(columns = {'turn' :stock_name})
    new_df.drop(['stock'],axis = 1,inplace = True)   
    return new_df

stock_names = turnover.stock.unique() # get individual stock id

old_df = pd.DataFrame() # initiallize 
for i in range(len(stock_names)):
    df = s.get_group(stock_names[i]) # find the individual stock data from 2011 to 2019
    if i == 0:
        new_df = stock_i(df)
        old_df = new_df
    else:
        new_df = stock_i(df)
        old_df = pd.merge(old_df,new_df,on = 'mddate',how = 'left')

old_df = old_df.set_index('mddate')

# 记得保存整理好的数据
old_df.to_csv(r'/Users/Zhang/Documents/华泰Project/数据/数据/raw_data/raw_data/Turnover_modified.csv')

<class 'numpy.ndarray'>


In [ ]:
# CHECKPOINT
# CHECKPOINT
# CHECKPOINT

## 个股最近N 个月内用每日换手率乘以每日收益率求算术平均值，N=1，3，6，12

In [72]:
os.chdir('/Users/Zhang/Documents/华泰Project/数据/数据/股票数据/')
def date(df):
    z = df.loc['2014-02-10':'2019-05-16']
    return z

turnover = date(read_data('turnoverRate'))
turnover_1 = slice(turnover)

def wgt_return_Nm(turnover_cal_Return,Return_cal_turnover,m):
    for i in range(300): #range(3622): # 3670
        wgt_return_list = []
        for j in range(200):#range(2013-m*20): # 2013-1*20
            turnover_array = np.array(turnover_cal_Return.iloc[j:m*20+j,i]).reshape(-1,1)
            return_array = np.array(Return_cal_turnover.iloc[j:m*20+j,i]).reshape(-1,1)
            wgt_return_list.append(np.nanmean(turnover_array*return_array))
            if i == 0:
                wgt_return_series = pd.Series(wgt_return_list)
                wgt_return_Df = wgt_return_series
            else:
                wgt_return_series = pd.Series(wgt_return_list)
                wgt_return_Df = pd.concat([wgt_return_Df,wgt_return_series],axis = 1)
    return wgt_return_Df

turnover_cal_Return = turnover_1.iloc[1:]
print(turnover_cal_Return.shape)
print(Return_cal_turnover.shape)
columns_1 = Stock_Return.columns.difference(turnover_cal_Return.columns)
Return_cal_turnover = Stock_Return.drop(columns_1,axis=1)

wgt_return_1m = wgt_return_Nm(turnover_cal_Return,Return_cal_turnover,1)
wgt_return_1m.to_csv(r'/Users/Zhang/Documents/华泰Project/Computed_Factor/Fundamental/wgt_return_1m')
#wgt_return_3m = wgt_return_Nm(turnover_cal_Return,Return_cal_turnover,3)
#wgt_return_3m.to_csv(r'/Users/Zhang/Documents/华泰Project/Computed_Factor/Fundamental/wgt_return_3m')
#wgt_return_6m = wgt_return_Nm(turnover_cal_Return,Return_cal_turnover,6)
#wgt_return_6m.to_csv(r'/Users/Zhang/Documents/华泰Project/Computed_Factor/Fundamental/wgt_return_6m')
#wgt_return_12m = wgt_return_Nm(turnover_cal_Return,Return_cal_turnover,12)
#wgt_return_12m.to_csv(r'/Users/Zhang/Documents/华泰Project/Computed_Factor/Fundamental/wgt_return_12m')

(1285, 300)
(63, 300)


/Users/Zhang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: Mean of empty slice


KeyboardInterrupt: 

## 动量反转
个股最近N个月内用每日换手率乘以函数exp(-x_i/N/4)再乘以每日
#收益率求算术平均值，x_i 为该日距离截面日的交易日的个数，N=1，3，6，12

#exp_wgt_return_Nm =

## 波动率

In [538]:
# 特质波动率——个股最近N 个月内用日频收益率对Fama French
# 三因子回归的残差的标准差，N=1，3，6，12
#std_FF3factor_Nm = 

7387710

## 计算股价

In [340]:
import numpy as np
std_list = []
In_Price = np.log(Price)
stock_array = Stock_Return.iloc[0:20,1]
std_m = np.std(stock_array)
std_list = std_list.append(std_m)
std_list = pd.Series(std_list)
print(std)

Series([], dtype: float64)


## 个股最近N 个月的日收益率序列标准差，N=1，3，6，12

In [350]:
def std_Nm(Stock_Return,m):
    std_record = pd.Series()
    for i in range(20): #range(3670): # 3670
        std_list = []
        for j in range(100): #range(2013-m*20): # 2013-60*20
            stock_array = np.array(Stock_Return.iloc[j:m*20+j,i]).reshape(-1,1)
            std_m = np.std(stock_array)
            std_list.append(std_m)
        std_series = pd.Series(std_list)
        std_record = pd.concat([std_record,std_series],axis = 1)
            
    return std_record

# 计算1个月日收益率序列标准差
std_1m = std_Nm(Stock_Return_1,1)

# 计算3个月日收益率序列标准差
#std_3m = std_Nm(Stock_Return,3)

# 计算6个月日收益率序列标准差
#std_6m = std_Nm(Stock_Return,6)

# 计算12个月日收益率序列标准差
#std_12m = std_Nm(Stock_Return,12)

In [504]:
# 因为使用Talib时候，input如果全为NaN，程序报错，所以先check array是否全为NaN
# 如果全为NaN，就不计算这个array
s = Price.iloc[:,1000].isnull().all()
print(s)
# check Price data if there is any column with all NaN values
count = 0
for i in range(Price.shape[1]):
    z = Price.iloc[:,i].isnull().all()
    if not z:
        pass
    else:
        count += 1
count


False


21

# 技术指标

In [1]:
import talib as ta
n = 20 # setup rollingwindow
rows = Price.shape[0]
cols = Price.shape[1]
MACD_DataFrame = pd.DataFrame()
# use pd dataframe or Series to concat

for i in range(cols):
    Nan = Price.iloc[:,i].isnull().all() #check if there is any NaN in column i
    
    if not Nan:
        Price_array = np.zeros((rows,))
        #Price_stock_name = Price.columns.values[i] # find the stock name
        Price_array = Price.values[:,i]
        MACD_series = pd.Series(ta.MACD(Price_array))#,timeperiod = n))
      
        #MACD_series.rename(Price_stock_name)
        if i == 0:
            MACD_DataFrame = MACD_series
        else:
            MACD_DataFrame = pd.concat([MACD_DataFrame,MACD_series],axis = 1)
    else:
        MACD_zeros = pd.Series(np.zeros((rows,)))
        MACD_DataFrame = pd.concat([MACD_DataFrame,MACD_zeros],axis = 1)

NameError: name 'Price' is not defined

In [523]:
def add_stock_name_date(df):
    df.columns = Price.columns
    date = Price.index
    df = df.set_index(date)
    return df
MACD_DataFrame = add_stock_name_date(MACD_DataFrame)

MACD_DataFrame.to_csv(r'/Users/Zhang/Documents/华泰Project/Computed_Factor/Technical/MACD_20.csv')


In [509]:
import talib as ta
n = 20 # setup rollingwindow
rows = Price.shape[0]
cols = Price.shape[1]
RSI_DataFrame = pd.DataFrame()
# use pd dataframe or Series to concat
for i in range(cols):
    Nan = Price.iloc[:,i].isnull().all() #check if there is any NaN in column i
    
    if not Nan:
        Price_array = np.zeros((rows,))
        #Price_stock_name = Price.columns.values[i] # find the stock name
        Price_array = Price.values[:,i]
        RSI_array = ta.RSI(Price_array,timeperiod = n)
        RSI_series = pd.Series(RSI_array)
        #RSI_series.rename(Price_stock_name)
        if i == 0:
            RSI_DataFrame = RSI_series
        else:
            RSI_DataFrame = pd.concat([RSI_DataFrame,RSI_series],axis = 1)
    else:
        RSI_zeros = pd.Series(np.zeros((rows,)))
        RSI_DataFrame = pd.concat([RSI_DataFrame,RSI_zeros],axis = 1)

#RSI_DataFrame 
RSI_DataFrame.columns = Price.columns
date = Price.index
RSI_DataFrame = RSI_DataFrame.set_index(date)
RSI_DataFrame.to_csv(r'/Users/Zhang/Documents/华泰Project/Computed_Factor/Technical/RSI_20.csv')


Index(['turn'], dtype='object')